In [1]:
from model import Model

Model.set_energyplus_folder("/usr/local/EnergyPlus-9-3-0/")

In [2]:
# Setup naive agent (Doing nothing, just calculate the reward based on the observation)
class Agent:
    def __init__(self):
        self.total_reward = dict()

    def step(self, state):
        if not self.total_reward:
            for key in state:
                self.total_reward[key] = 0
        self.reward(state)
        print(state)

    def reward(self, state):
        for i, key in enumerate(state):
            if isinstance(state[key], dict):
                counter = 0
                average_value = 0
                for _, value in state[key].items():
                    average_value += value
                    counter += 1
                self.total_reward[key] += average_value / counter
            else:
                self.total_reward[key] += state[key]

In [3]:
# Initialize the model
naive_agent = Agent()
model = Model(idf_file_name="./buildings/5ZoneAirCooled.idf",
              weather_file="./weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw",
              agent=naive_agent)

In [4]:
# Example of how to check the required fields for a new component
print(model.get_sub_configuration("Output:Variable"))

['key', 'Key_Value', 'Variable_Name', 'Reporting_Frequency', 'Schedule_Name']


In [5]:
# Setup values for new components
occu_schedule_values = {"Name": "Test_Schedule",
                        "Schedule Type Limits Name": "Fraction",
                        "Field 1": "Through: 12/31",
                        "Field 2": "For: Alldays",
                        "Field 3": "Until: 05:00",
                        "Field 4": "0",
                        "Field 5": "Until 09:00",
                        "Field 6": "0.5",
                        "Field 7": "Until 17:00",
                        "Field 8": "0.95",
                        "Field 9": "Until 24:00",
                        "Field 10": "0.5"}

activity_values = {"Name": "Test_Activity_Schedule",
                   "Schedule Type Limits Name": "Any Number",
                   "Field 1": "Through:12/31",
                   "Field 2": "For: Alldays",
                   "Field 3": "Until 24:00",
                   "Field 4": "117"}

work_efficiency = {"Name": "Test_Work_Schedule",
                   "Schedule Type Limits Name": "Fraction",
                   "Field 1": "Through:12/31",
                   "Field 2": "For: Alldays",
                   "Field 3": "Until 24:00",
                   "Field 4": "0.1"}

cloth_schedule = {"Name": "Test_Cloth_Schedule",
                  "Schedule Type Limits Name": "Fraction",
                  "Field 1": "Through:12/31",
                  "Field 2": "For: Alldays",
                  "Field 3": "Until 24:00",
                  "Field 4": "0.9"}

air_velocity = {"Name": "Test_Air_Velocity",
                "Schedule Type Limits Name": "Fraction",
                "Field 1": "Through:12/31",
                "Field 2": "For: Alldays",
                "Field 3": "Until 24:00",
                "Field 4": "0.25"}

people_values = {"Name": "Test",
                 "Zone or ZoneList Name": "SPACE1-1",
                 "Number of People Schedule Name": "Test_Schedule",
                 "Number of People": 5,
                 "Activity Level Schedule Name": "Test_Activity_Schedule",
                 "Work Efficiency Schedule Name": "Test_Work_Schedule",
                 "Clothing Insulation Schedule Name": "Test_Cloth_Schedule",
                 "Air Velocity Schedule Name": "Test_Air_Velocity",
                 "Thermal Comfort Model 1 Type": "Fanger"}

In [6]:
print(model.add_configuration("Schedule:Compact", values=occu_schedule_values))


SCHEDULE:COMPACT,
    Test_Schedule,            !- Name
    Fraction,                 !- Schedule Type Limits Name
    Through: 12/31,           !- Field 1
    For: Alldays,             !- Field 2
    Until: 05:00,             !- Field 3
    0,                        !- Field 4
    Until 09:00,              !- Field 5
    0.5,                      !- Field 6
    Until 17:00,              !- Field 7
    0.95,                     !- Field 8
    Until 24:00,              !- Field 9
    0.5;                      !- Field 10



In [7]:
print(model.add_configuration("Schedule:Compact", values=activity_values))


SCHEDULE:COMPACT,
    Test_Activity_Schedule,    !- Name
    Any Number,               !- Schedule Type Limits Name
    Through:12/31,            !- Field 1
    For: Alldays,             !- Field 2
    Until 24:00,              !- Field 3
    117;                      !- Field 4



In [8]:
print(model.add_configuration("Schedule:Compact", values=work_efficiency))


SCHEDULE:COMPACT,
    Test_Work_Schedule,       !- Name
    Fraction,                 !- Schedule Type Limits Name
    Through:12/31,            !- Field 1
    For: Alldays,             !- Field 2
    Until 24:00,              !- Field 3
    0.1;                      !- Field 4



In [9]:
print(model.add_configuration("Schedule:Compact", values=cloth_schedule))


SCHEDULE:COMPACT,
    Test_Cloth_Schedule,      !- Name
    Fraction,                 !- Schedule Type Limits Name
    Through:12/31,            !- Field 1
    For: Alldays,             !- Field 2
    Until 24:00,              !- Field 3
    0.9;                      !- Field 4



In [10]:
print(model.add_configuration("Schedule:Compact", values=air_velocity))


SCHEDULE:COMPACT,
    Test_Air_Velocity,        !- Name
    Fraction,                 !- Schedule Type Limits Name
    Through:12/31,            !- Field 1
    For: Alldays,             !- Field 2
    Until 24:00,              !- Field 3
    0.25;                     !- Field 4



In [11]:
# Add some occupancy into the model
print(model.add_configuration("People", values=people_values))


PEOPLE,
    Test,                     !- Name
    SPACE1-1,                 !- Zone or ZoneList Name
    Test_Schedule,            !- Number of People Schedule Name
    People,                   !- Number of People Calculation Method
    5,                        !- Number of People
    ,                         !- People per Zone Floor Area
    ,                         !- Zone Floor Area per Person
    0.3,                      !- Fraction Radiant
    autocalculate,            !- Sensible Heat Fraction
    Test_Activity_Schedule,    !- Activity Level Schedule Name
    3.82e-08,                 !- Carbon Dioxide Generation Rate
    No,                       !- Enable ASHRAE 55 Comfort Warnings
    ZoneAveraged,             !- Mean Radiant Temperature Calculation Type
    ,                         !- Surface NameAngle Factor List Name
    Test_Work_Schedule,       !- Work Efficiency Schedule Name
    ClothingInsulationSchedule,    !- Clothing Insulation Calculation Method
    ,       

In [12]:
# Add a thermal comfort model to the model
print(model.add_configuration("Output:Variable", values={"Variable Name": "Zone Thermal Comfort Fanger Model PMV",
                                                         "Reporting_Frequency": "timestep"}))


OUTPUT:VARIABLE,
    *,                        !- Key Value
    Zone Thermal Comfort Fanger Model PMV,    !- Variable Name
    timestep;                 !- Reporting Frequency



In [13]:
# Example of check what is available for the state value
print(model.get_available_names_under_group("Output:Variable"))

['Site Outdoor Air Drybulb Temperature', 'Zone Air Temperature', 'Zone Mean Air Dewpoint Temperature', 'Zone Air System Sensible Cooling Rate', 'Zone Air System Sensible Heating Rate', 'System Node Temperature', 'System Node Mass Flow Rate', 'System Node Setpoint Temperature', 'System Node Humidity Ratio', 'System Node Temperature', 'System Node Mass Flow Rate', 'System Node Setpoint Temperature', 'System Node Humidity Ratio', 'System Node Temperature', 'System Node Humidity Ratio', 'System Node Dewpoint Temperature', 'System Node Temperature', 'System Node Mass Flow Rate', 'System Node Humidity Ratio', 'System Node Setpoint Temperature', 'System Node Temperature', 'System Node Mass Flow Rate', 'System Node Mass Flow Rate', 'System Node Temperature', 'System Node Mass Flow Rate', 'System Node Mass Flow Rate', 'Heating Coil Heating Rate', 'Cooling Coil Total Cooling Rate', 'Cooling Coil Sensible Cooling Rate', 'Plant Supply Side Cooling Demand Rate', 'Plant Supply Side Heating Demand Ra

In [14]:
model.simulate()

{'temperature': {'PLENUM-1': 23.0, 'SPACE1-1': 23.0, 'SPACE2-1': 23.0, 'SPACE3-1': 23.0, 'SPACE4-1': 23.0, 'SPACE5-1': 23.0}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 22.874110015900605, 'SPACE1-1': 22.881461232729034, 'SPACE2-1': 22.8848287850367, 'SPACE3-1': 22.88775149729128, 'SPACE4-1': 22.88205046255434, 'SPACE5-1': 22.98096358783965}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 22.784326585624097, 'SPACE1-1': 22.724117800155636, 'SPACE2-1': 22.75280281188858, 'SPACE3-1': 22.736517734419994, 'SPACE4-1': 22.749815467626114, 'SPACE5-1': 22.966812132444264}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 22.733483849791796, 'SPACE1-1': 22.571542499703828, 'SPACE2-1': 22.636423270209367, 'SPACE3-1': 22.588328879821606, 'SPACE4-1': 22.63514666703679, 'SPACE5-1': 22.95720997986774}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 22.695073652851356, 'SPACE1-1': 22.435867825529687, 'SP

{'temperature': {'PLENUM-1': 16.906612702322434, 'SPACE1-1': 17.375852697693155, 'SPACE2-1': 17.63106402377051, 'SPACE3-1': 17.41697795524427, 'SPACE4-1': 17.631333990401473, 'SPACE5-1': 19.654355017214158}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 16.88722249633327, 'SPACE1-1': 17.365740783310493, 'SPACE2-1': 17.619871210332267, 'SPACE3-1': 17.406930509320432, 'SPACE4-1': 17.62000830269018, 'SPACE5-1': 19.645447617371588}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 16.867912225492827, 'SPACE1-1': 17.3556794214464, 'SPACE2-1': 17.608733173842875, 'SPACE3-1': 17.39693360886393, 'SPACE4-1': 17.60873762520636, 'SPACE5-1': 19.63657845483171}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 16.848689644266784, 'SPACE1-1': 17.34566846887122, 'SPACE2-1': 17.59764971441879, 'SPACE3-1': 17.386987107477147, 'SPACE4-1': 17.597521758773297, 'SPACE5-1': 19.62774752616452}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'te

{'temperature': {'PLENUM-1': 14.66817848103269, 'SPACE1-1': 16.70004638037313, 'SPACE2-1': 16.70021979889631, 'SPACE3-1': 16.700062834487053, 'SPACE4-1': 16.700139607534343, 'SPACE5-1': 18.716742790747347}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 14.661683241544633, 'SPACE1-1': 16.700055577126896, 'SPACE2-1': 16.699743680820845, 'SPACE3-1': 16.700025353567653, 'SPACE4-1': 16.699941079729907, 'SPACE5-1': 18.71381242260283}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 14.655223790486517, 'SPACE1-1': 16.700040185923754, 'SPACE2-1': 16.699941012925862, 'SPACE3-1': 16.699991639740862, 'SPACE4-1': 16.699829537238987, 'SPACE5-1': 18.710896658573752}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 14.648804832484565, 'SPACE1-1': 16.700078200510262, 'SPACE2-1': 16.699919056624733, 'SPACE3-1': 16.699994810762142, 'SPACE4-1': 16.70007132091093, 'SPACE5-1': 18.707995447544015}, 'electricity': 0.0, 'PMV': {'Test': 0.0

{'temperature': {'PLENUM-1': 14.805014181116256, 'SPACE1-1': 18.1655598769798, 'SPACE2-1': 20.874609996709538, 'SPACE3-1': 16.899314176443742, 'SPACE4-1': 16.897990763029437, 'SPACE5-1': 18.544483015308085}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 14.814215281096823, 'SPACE1-1': 18.182234607450205, 'SPACE2-1': 20.891882795937637, 'SPACE3-1': 16.903610054079678, 'SPACE4-1': 16.902033749892723, 'SPACE5-1': 18.54551344374991}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 14.823528486606032, 'SPACE1-1': 18.199149601158002, 'SPACE2-1': 20.909105360018756, 'SPACE3-1': 16.90806860214952, 'SPACE4-1': 16.906203370369003, 'SPACE5-1': 18.546752353081352}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 14.8329424444357, 'SPACE1-1': 18.216271825438696, 'SPACE2-1': 20.926215948563737, 'SPACE3-1': 16.912660961925866, 'SPACE4-1': 16.910477201769286, 'SPACE5-1': 18.548168977735802}, 'electricity': 0.0, 'PMV': {'Test': 0.0}

{'temperature': {'PLENUM-1': 16.923702977001977, 'SPACE1-1': 19.775888355516436, 'SPACE2-1': 19.462644976092264, 'SPACE3-1': 18.177499292575195, 'SPACE4-1': 18.272370101601094, 'SPACE5-1': 19.28567883636153}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 16.927545494297092, 'SPACE1-1': 19.774193557560586, 'SPACE2-1': 19.452616639117036, 'SPACE3-1': 18.180052864703555, 'SPACE4-1': 18.275195213216527, 'SPACE5-1': 19.288181262905507}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 16.93133156297547, 'SPACE1-1': 19.772548432296645, 'SPACE2-1': 19.442639569822695, 'SPACE3-1': 18.18254565428647, 'SPACE4-1': 18.277958677435123, 'SPACE5-1': 19.290669471840516}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}
{'temperature': {'PLENUM-1': 16.935067753899585, 'SPACE1-1': 19.770951248981117, 'SPACE2-1': 19.432715317906016, 'SPACE3-1': 18.18497941591352, 'SPACE4-1': 18.28066199622317, 'SPACE5-1': 19.2931434671623}, 'electricity': 0.0, 'PMV': {'Test': 0.0}}

...

{'temperature': {'PLENUM-1': 19.52815373079599, 'SPACE1-1': 22.2002735882264, 'SPACE2-1': 22.200530542239548, 'SPACE3-1': 22.200298632339468, 'SPACE4-1': 22.200593513541754, 'SPACE5-1': 23.4308428633982}, 'electricity': 721885.7651259899, 'PMV': {'Test': -0.934707648748658}}
{'temperature': {'PLENUM-1': 19.54122094751227, 'SPACE1-1': 22.200228690839964, 'SPACE2-1': 22.200238223173038, 'SPACE3-1': 22.200160150462004, 'SPACE4-1': 22.19996447015322, 'SPACE5-1': 23.438911838182232}, 'electricity': 721883.0479446934, 'PMV': {'Test': -0.933860646155385}}
{'temperature': {'PLENUM-1': 19.554204125557906, 'SPACE1-1': 22.20038095251236, 'SPACE2-1': 22.199986933357053, 'SPACE3-1': 22.200309033911278, 'SPACE4-1': 22.200233057615165, 'SPACE5-1': 23.446874707669167}, 'electricity': 721881.5331880149, 'PMV': {'Test': -0.9330370568835614}}
{'temperature': {'PLENUM-1': 19.56718782152079, 'SPACE1-1': 22.200168523821755, 'SPACE2-1': 22.200393552657065, 'SPACE3-1': 22.20025991309087, 'SPACE4-1': 22.200254

{'temperature': {'PLENUM-1': 21.198995106268033, 'SPACE1-1': 22.359141914731683, 'SPACE2-1': 22.19985638205323, 'SPACE3-1': 22.19999513441481, 'SPACE4-1': 22.199919292395276, 'SPACE5-1': 23.613728082074516}, 'electricity': 654210.2110265577, 'PMV': {'Test': -0.8527963460985362}}
{'temperature': {'PLENUM-1': 21.20562922303792, 'SPACE1-1': 22.362864576708677, 'SPACE2-1': 22.200044284836242, 'SPACE3-1': 22.20007740707228, 'SPACE4-1': 22.200276935056376, 'SPACE5-1': 23.616556773080827}, 'electricity': 654210.4182540564, 'PMV': {'Test': -0.8521386896789063}}
{'temperature': {'PLENUM-1': 21.212262686890462, 'SPACE1-1': 22.366625992314994, 'SPACE2-1': 22.200388109618295, 'SPACE3-1': 22.20002859906061, 'SPACE4-1': 22.200422270970527, 'SPACE5-1': 23.61943844615528}, 'electricity': 654209.560321316, 'PMV': {'Test': -0.8514667637615595}}
{'temperature': {'PLENUM-1': 21.218895536935307, 'SPACE1-1': 22.370426022653593, 'SPACE2-1': 22.199911356365785, 'SPACE3-1': 22.200042795024125, 'SPACE4-1': 22.2

{'temperature': {'PLENUM-1': 22.170451571529085, 'SPACE1-1': 22.49843576968297, 'SPACE2-1': 22.200400220997665, 'SPACE3-1': 22.20023836191544, 'SPACE4-1': 22.200075987715177, 'SPACE5-1': 23.899882808036025}, 'electricity': 684369.4410887986, 'PMV': {'Test': -0.766492088602425}}
{'temperature': {'PLENUM-1': 22.169618690147207, 'SPACE1-1': 22.49358606298865, 'SPACE2-1': 22.200429595563456, 'SPACE3-1': 22.20023276712437, 'SPACE4-1': 22.200225083629928, 'SPACE5-1': 23.899882863978796}, 'electricity': 684367.1332948207, 'PMV': {'Test': -0.7674972284823454}}
{'temperature': {'PLENUM-1': 22.168731082243262, 'SPACE1-1': 22.488847128281538, 'SPACE2-1': 22.200352753834537, 'SPACE3-1': 22.200092266190882, 'SPACE4-1': 22.200302368736715, 'SPACE5-1': 23.899882931070444}, 'electricity': 684364.6177269793, 'PMV': {'Test': -0.7685007390997}}
{'temperature': {'PLENUM-1': 22.167815828532948, 'SPACE1-1': 22.484226580994026, 'SPACE2-1': 22.200309487727083, 'SPACE3-1': 22.20007146695108, 'SPACE4-1': 22.199

{'temperature': {'PLENUM-1': 21.463713202755795, 'SPACE1-1': 22.200564167806455, 'SPACE2-1': 22.200395824308526, 'SPACE3-1': 22.200412840850674, 'SPACE4-1': 22.200786397054706, 'SPACE5-1': 23.12058694864379}, 'electricity': 384699.4808114055, 'PMV': {'Test': -0.8984019747320446}}
{'temperature': {'PLENUM-1': 21.441225825971866, 'SPACE1-1': 22.20067445731866, 'SPACE2-1': 22.200784195026582, 'SPACE3-1': 22.20045661961923, 'SPACE4-1': 22.200374157885474, 'SPACE5-1': 23.056474154154476}, 'electricity': 384703.1363344195, 'PMV': {'Test': -0.8997117615736904}}
{'temperature': {'PLENUM-1': 21.422533096146662, 'SPACE1-1': 22.20062850113659, 'SPACE2-1': 22.200589417566285, 'SPACE3-1': 22.20067267591191, 'SPACE4-1': 22.200665578560574, 'SPACE5-1': 22.998741044022914}, 'electricity': 384705.2058589058, 'PMV': {'Test': -0.9009558559964794}}
{'temperature': {'PLENUM-1': 21.406070348872817, 'SPACE1-1': 22.200431490843197, 'SPACE2-1': 22.200310932505435, 'SPACE3-1': 22.20078825010033, 'SPACE4-1': 22.

{'temperature': {'PLENUM-1': 19.089239890528177, 'SPACE1-1': 19.30330913552923, 'SPACE2-1': 19.337106919661867, 'SPACE3-1': 19.098895824953747, 'SPACE4-1': 19.344435641296567, 'SPACE5-1': 20.349492710211663}, 'electricity': 119267.20144408869, 'PMV': {'Test': -1.5693057964810684}}
{'temperature': {'PLENUM-1': 19.07546283784583, 'SPACE1-1': 19.291993594392018, 'SPACE2-1': 19.324547263893002, 'SPACE3-1': 19.086747571534705, 'SPACE4-1': 19.331888685309412, 'SPACE5-1': 20.33956811215426}, 'electricity': 119267.20144408869, 'PMV': {'Test': -1.572127450489378}}
{'temperature': {'PLENUM-1': 19.061757644445933, 'SPACE1-1': 19.280819113560828, 'SPACE2-1': 19.312129110606385, 'SPACE3-1': 19.074757705575507, 'SPACE4-1': 19.319481834107698, 'SPACE5-1': 20.329758411528154}, 'electricity': 119267.20144408869, 'PMV': {'Test': -1.574912641775451}}
{'temperature': {'PLENUM-1': 19.048127793816587, 'SPACE1-1': 19.269782739881148, 'SPACE2-1': 19.299849924706372, 'SPACE3-1': 19.06292270832537, 'SPACE4-1': 

In [15]:
# Check final reward, it can be used to calculate the Q values etc.
print(naive_agent.total_reward)

{'temperature': 206262.86501267794, 'electricity': 634103034.7600744, 'PMV': -4859.971877974967}
